In [1]:
import os
import json
import re
from string import printable
import pandas as pd

In [2]:
def get_file_list(datadir='../../news-crawler/data/articles/'):
  files = []
  for dirpath, dirnames, filenames in os.walk(datadir):
    if len(filenames) > 0:
      files.extend([os.path.join(dirpath, x) for x in filenames])
  return files

def file_generator():
  files = get_file_list()
  for filename in files:
    with open(filename, 'r') as f:
      yield json.load(f)
      
def clean_string(string):
  string = re.sub('\s+', ' ', string)
  string = ''.join([s for s in string if s in printable])
  return string

In [3]:
out_text = 'politics-articles.txt'
out_label = 'politics-labels.txt'
out_url = 'politics-urls.txt'

for filename in [out_text, out_label, out_url]:
  if os.path.exists(filename):
    os.remove(filename)

for data in file_generator():
  if data['label'] == 'satirical':
    continue
  if len(data['taxonomy']) == 0:
    continue
  labels = [data['taxonomy'][i]['label'] for i in range(len(data['taxonomy']))]
  labels = [x for x in labels if re.match('.*politics', x)]
  if len(labels) == 0:
    continue
  with open(out_text, 'a') as g:
    g.write('{}\n'.format(clean_string(data['text'])))
  with open(out_label, 'a') as g:
    g.write('{}\n'.format(data['label']))
  with open(out_url, 'a') as g:
    g.write('{}\n'.format(data['url']))